In [19]:
import spacy
import pandas as pd
import multiprocessing
import numpy as np
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string

In [20]:
df_hansard  = pd.read_csv('data/hansard_all.csv')
df_hansard.head()
df_group = df_hansard.groupby('subjectOfBusinessId')


In [21]:
# df_hansard  = pd.read_csv('data/hansard_all.csv')

# df_group = df_hansard.groupby('subjectOfBusinessId')

q_a = []
for i, index in df_group.groups.items():
    # don't bother with odd pairs
    if (len(index) % 2 != 0): 
        continue

    # Create conversatoin pairs
    t = df_hansard.iloc[list(index)]['content'].values
    
    temp = list(zip(t[::2], t[1::2], 
                    df_hansard.iloc[index]['SubjectOfBusinessTitle'],
                    df_hansard.iloc[index]['FloorLanguage'],
                    df_hansard.iloc[index]['date'],
                    df_hansard.iloc[index]['personSpeaking'],                
                   ))
#     temp.append(df_hansard.iloc[index]['SubjectOfBusinessTitle'])
#     print(list(zip(t[::2], t[1::2])))
#     print(df_hansard.iloc[list(index)]['SubjectOfBusinessTitle'])
#     print(temp)
    q_a.append(temp)
#     break

q_a = [item for sublist in q_a for item in sublist]

# print(q_a[0])
# q_a = q_a[:10000]

print('number of q & a', len(q_a))
    
df_q_a = pd.DataFrame(q_a)
df_q_a.columns = ['Q', 'A', 'SubjectOfBusinessTitle', 'FloorLanguage', 'date', 'personSpeaking']
df_q_a.to_csv('data/q_a_all.csv')
df_q_a.tail()

number of q & a 27732


,Q,A,SubjectOfBusinessTitle,FloorLanguage,date,personSpeaking
27727,"\n Mr. Speaker, the Prime Ministe...","\n Mr. Speaker, of course, at no ...",Public Safety,EN,2018-03-27,"Right Hon. Justin Trudeau (Prime Minister, Lib.)"
27728,"\n Mr. Speaker, the only reason w...","\n Mr. Speaker, only Stephen Harp...",Public Safety,EN,2018-03-27,"Hon. Andrew Scheer (Leader of the Opposition, ..."
27729,"\n Mr. Speaker, in a healthy and ...","\n Mr. Speaker, I thank my collea...",Status of Women,EN,2018-03-27,"Mrs. Eva Nassif (Vimy, Lib.)"
27730,"\n Mr. Speaker, the government ha...","\n Mr. Speaker, I was at the comm...",Status of Women,EN,2018-03-27,"Hon. Lisa Raitt (Milton, CPC)"
27731,"\n Mr. Speaker, there is no attac...","\n Mr. Speaker, we often see in t...",Status of Women,EN,2018-03-27,Mr. Joël Lightbound (Parliamentary Secretary t...


In [22]:
# df_hansard  = pd.read_csv('data/hansard_all.csv')

# df_group = df_hansard.groupby('subjectOfBusinessId')

# q_a = []
# for i, index in df_group.groups.items():
#     # don't bother with odd pairs
#     if (len(index) % 2 != 0): 
#         continue

#     # Create conversatoin pairs
#     t = df_hansard.iloc[list(index)]['content'].values
#     q_a.append(list(zip(t[::2], t[1::2])))

# q_a = [item for sublist in q_a for item in sublist]

# # q_a = q_a[:10000]

# print('number of q & a', len(q_a))
    
# df_q_a = pd.DataFrame(q_a)
# df_q_a.columns = ['Q', 'A']
# df_q_a.to_csv('data/q_a_all.csv')
# df_q_a.tail()

In [23]:
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load('en')

def sentence_tokenizer(raw_text):
    # Create doc = nlp(raw_text) and parse sentences. 
    return u' '.join([sent.string.strip() for sent in nlp(raw_text).sents])


# print(list(nlp(df_q_a['Q'][2]).sents))
# print(df_q_a['Q'][2])
# print(sentence_tokenizer(df_q_a['Q'][2]))

In [24]:
%%time

def _apply_df(args):
    df, func, kwargs = args
    df['Q'] = df['Q'].apply(func, **kwargs)
    df['A'] = df['A'].apply(func, **kwargs)
    return df#df.apply(func, **kwargs)

def apply_by_multiprocessing(df, func, **kwargs):
    workers = kwargs.pop('workers')
    pool = multiprocessing.Pool(processes=workers)
    result = pool.map(_apply_df, [(d, func, kwargs) for d in np.array_split(df, workers)])
    pool.close()
    return pd.concat(list(result))
    



CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 12.6 µs


In [25]:
%%time

num_cores = multiprocessing.cpu_count()
print(num_cores) 
    
df_q_a = pd.read_csv('data/q_a_all.csv')

df_q_a = apply_by_multiprocessing(df_q_a, sentence_tokenizer,  workers=num_cores)
   
df_q_a.to_csv('data/q_a_all.csv')


# df_q_a = pd.read_csv('data/q_a_all.csv')
# df_q_a.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
# df_q_a.to_csv('data/q_a_all.csv')
# text= ' Mr. Speaker, when the Prime Minister interrupted a woman at a town hall, correcting her use of “mankind” with “peoplekind“, his mansplaining went viral. Around the world, the Prime Minister was mocked for his political correctness. The Prime Minister eventually conceded that it was a dumb joke, but his principal secretary, Gerald Butts, tweeted that any and all who criticized his boss were Nazis. The Prime Minister once said that any statement by Mr. Butts could be considered his own, and in this case?'
# print(df_q_a['Q'][10])
# print(sentence_tokenizer(df_q_a['Q'][10]))

# df_q_a['Q'] = df_q_a['Q'].apply(sentence_tokenizer)
# df_q_a['A'] = df_q_a['A'].apply(sentence_tokenizer)
# df_q_a.to_csv('data/q_a.csv')
# df_q_a.tail()



8
CPU times: user 2.12 s, sys: 352 ms, total: 2.48 s
Wall time: 23min 1s


In [26]:
df_q_a.tail()

,Unnamed: 0,Q,A,SubjectOfBusinessTitle,FloorLanguage,date,personSpeaking
27727,27727,"Mr. Speaker, the Prime Minister allowed a conv...","Mr. Speaker, of course, at no point did member...",Public Safety,EN,2018-03-27,"Right Hon. Justin Trudeau (Prime Minister, Lib.)"
27728,27728,"Mr. Speaker, the only reason why the Prime Min...","Mr. Speaker, only Stephen Harper's Conservativ...",Public Safety,EN,2018-03-27,"Hon. Andrew Scheer (Leader of the Opposition, ..."
27729,27729,"Mr. Speaker, in a healthy and prosperous socie...","Mr. Speaker, I thank my colleague from Vimy fo...",Status of Women,EN,2018-03-27,"Mrs. Eva Nassif (Vimy, Lib.)"
27730,27730,"Mr. Speaker, the government has a troubling hi...","Mr. Speaker, I was at the committee meeting ye...",Status of Women,EN,2018-03-27,"Hon. Lisa Raitt (Milton, CPC)"
27731,27731,"Mr. Speaker, there is no attack in saying the ...","Mr. Speaker, we often see in the House a type ...",Status of Women,EN,2018-03-27,Mr. Joël Lightbound (Parliamentary Secretary t...


In [27]:
df_q_a['Q_A'] = df_q_a[['Q', 'A']].apply(lambda x: u'\n'.join(x), axis=1)


print(len(df_q_a['Q_A']))
with open('data/Q_A_pairs.txt', 'w') as file:
    for row in df_q_a['Q_A']:
        file.write(row.lower().strip() + '\n')


27732


In [11]:
df_q_a = pd.read_csv('data/q_a_test.csv')

import re
import nltk

# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def text_to_sents(text_seqs):
    tokens = normalizeString(unicodeToAscii( text_seqs))
    return nltk.sent_tokenize(tokens) # this gives us a list of sentences

df_q_a['Q_test'] = df_q_a['Q'].apply(text_to_sents)
df_q_a['Q_test']
df_q_a.to_csv('data/q_a_test.csv')